In [1]:
# import modules

import json
import pickle
import spacy
import string
import warnings
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random
nlp = spacy.load("en_core_web_sm")

Using TensorFlow backend.


In [2]:
#initializing lists and variables
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)
type(words)

list

In [3]:
#preparing data
#converting user response to spacy token
def responsetotoken(user_response):
    return nlp(user_response)



#function to word tokenize and lower case
def lower_token(text):
  return [token.lower_ for token in text]

for intent in intents['intents']:
    for pattern in intent['patterns']:

        # converting pattern to spacy token
        pattern=nlp(pattern)
        w=lower_token(pattern)
        # take each word and tokenize it

        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])


In [4]:
#preprocessing 
## lemmatized lowercase words after removing punctuations with SPACY
def LemNormalize(text):
    token_text=responsetotoken(text)
    return [token.lower_ for token in token_text if not token.is_stop and not token.is_punct]

corpus=[]

for w in words:
    norm_word=(LemNormalize(w))
    corpus.extend(norm_word)


words = sorted(list(set(corpus)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)

92 documents
19 classes [' Health and safety', 'assessment', 'business', 'cases', 'cases in canada', 'cerb', 'coronavirus', 'ei', 'goodbye', 'greeting', 'individual', 'options', 'prevention', 'support', 'symptoms', 'thanks', 'travellers', 'update', 'video']
84 unique lemmatized words [' ', 'affect', 'apply', 'area', 'assessment', 'awesome', 'benefits', 'benfits', 'busines', 'business', 'bye', 'canada', 'canadians', 'cases', 'cerb', 'chatting', 'confirmed', 'convid-19', 'coronavirus', 'count', 'country', 'covid-19', 'day', 'deaths', 'determinte', 'device', 'disease', 'drug', 'economic', 'ei', 'emergencey', 'employee', 'financial', 'good', 'goodbye', 'health', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hola', 'individual', 'individuals', 'insurance', 'later', 'live', 'measures', 'medical', 'monitoring', 'new', 'nice', 'notice', 'novel', 'offered', 'outbreak', 'pandameic', 'prepared', 'prevent', 'prevention', 'preventions', 'provide', 'response', 'risk', 'risks', 'safety', 'sear

In [5]:
#saving pickle file


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [6]:
## lemmatized lowercase words after removing punctuations with SPACY
def LemNormalize_train(text):
    token_text=responsetotoken(text)
    return [token.lower_ for token in token_text if  not token.is_punct]
print(LemNormalize_train('hi'))
print(LemNormalize_train('there'))


['hi']
['there']


In [7]:
# initializing training data
training = []
output_empty = [0] * len(classes)

for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    #empty list for lemmatizing words in doc
    corpus=[]
    for w in pattern_words:
        norm_word=(LemNormalize_train(w))
        corpus.extend(norm_word)

    pattern_words = corpus
    print("pattern words:",pattern_words)
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
    corpus=[]
    

pattern words: ['hi', 'there']
pattern words: ['how', 'are', 'you']
pattern words: ['is', 'anyone', 'there']
pattern words: ['hey']
pattern words: ['hola']
pattern words: ['hello']
pattern words: ['good', 'day']
pattern words: ['bye']
pattern words: ['see', 'you', 'later']
pattern words: ['goodbye']
pattern words: ['thanks', 'see', 'you', 'later']
pattern words: ['nice', 'chatting', 'to', 'you', 'bye']
pattern words: ['till', 'next', 'time']
pattern words: ['thanks']
pattern words: ['thank', 'you']
pattern words: ['that', "'s", 'helpful']
pattern words: ['awesome', 'thanks']
pattern words: ['thanks', 'for', 'helping', 'me']
pattern words: ['how', 'you', 'could', 'help', 'me']
pattern words: ['what', 'you', 'can', 'do']
pattern words: ['what', 'help', 'you', 'provide']
pattern words: ['how', 'you', 'can', 'be', 'helpful']
pattern words: ['what', 'support', 'is', 'offered']
pattern words: ['support']
pattern words: ['financial', 'support']
pattern words: ['economic', 'support']
pattern w

In [8]:
print(output_row)
print(pattern_words)

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['travel', 'safety']


In [9]:
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")
print(training)

Training data created
[[list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0])
  list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0])]
 [list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
  list([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])]
 [list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
  list([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [10]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))



Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [11]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])



In [12]:
#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model_1.h5', hist)

print("model created")


Epoch 1/200
92/92 [==============================] - 0s 4ms/step - loss: 2.9467 - accuracy: 0.0326
Epoch 2/200
92/92 [==============================] - 0s 779us/step - loss: 2.9261 - accuracy: 0.1413
Epoch 3/200
92/92 [==============================] - 0s 629us/step - loss: 2.8453 - accuracy: 0.1087
Epoch 4/200
92/92 [==============================] - 0s 693us/step - loss: 2.8401 - accuracy: 0.1087
Epoch 5/200
92/92 [==============================] - 0s 738us/step - loss: 2.7864 - accuracy: 0.1196
Epoch 6/200
92/92 [==============================] - 0s 770us/step - loss: 2.7088 - accuracy: 0.1522
Epoch 7/200
92/92 [==============================] - 0s 643us/step - loss: 2.6408 - accuracy: 0.1739
Epoch 8/200
92/92 [==============================] - 0s 750us/step - loss: 2.5552 - accuracy: 0.2065
Epoch 9/200
92/92 [==============================] - 0s 750us/step - loss: 2.4802 - accuracy: 0.2174
Epoch 10/200
92/92 [==============================] - 0s 1ms/step - loss: 2.5024 - accuracy:

92/92 [==============================] - 0s 724us/step - loss: 0.4553 - accuracy: 0.8587
Epoch 80/200
92/92 [==============================] - 0s 728us/step - loss: 0.3096 - accuracy: 0.9130
Epoch 81/200
92/92 [==============================] - 0s 576us/step - loss: 0.2988 - accuracy: 0.9239
Epoch 82/200
92/92 [==============================] - 0s 662us/step - loss: 0.5165 - accuracy: 0.8152
Epoch 83/200
92/92 [==============================] - 0s 666us/step - loss: 0.3009 - accuracy: 0.9022
Epoch 84/200
92/92 [==============================] - 0s 773us/step - loss: 0.2874 - accuracy: 0.9022
Epoch 85/200
92/92 [==============================] - 0s 633us/step - loss: 0.3190 - accuracy: 0.9130
Epoch 86/200
92/92 [==============================] - 0s 743us/step - loss: 0.3945 - accuracy: 0.8696
Epoch 87/200
92/92 [==============================] - 0s 666us/step - loss: 0.5308 - accuracy: 0.8152
Epoch 88/200
92/92 [==============================] - 0s 588us/step - loss: 0.2693 - accuracy: 

92/92 [==============================] - 0s 609us/step - loss: 0.2751 - accuracy: 0.9022
Epoch 159/200
92/92 [==============================] - 0s 674us/step - loss: 0.1564 - accuracy: 0.9457
Epoch 160/200
92/92 [==============================] - 0s 688us/step - loss: 0.2625 - accuracy: 0.8913
Epoch 161/200
92/92 [==============================] - 0s 715us/step - loss: 0.1833 - accuracy: 0.9239
Epoch 162/200
92/92 [==============================] - 0s 668us/step - loss: 0.2458 - accuracy: 0.9348
Epoch 163/200
92/92 [==============================] - 0s 649us/step - loss: 0.2230 - accuracy: 0.9239
Epoch 164/200
92/92 [==============================] - 0s 797us/step - loss: 0.2125 - accuracy: 0.9239
Epoch 165/200
92/92 [==============================] - 0s 524us/step - loss: 0.2160 - accuracy: 0.9239
Epoch 166/200
92/92 [==============================] - 0s 488us/step - loss: 0.2329 - accuracy: 0.9022
Epoch 167/200
92/92 [==============================] - 0s 513us/step - loss: 0.2817 - a